<a href="https://colab.research.google.com/github/FullteaOfEEIC/er/blob/master/er.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#seq2seq
!git clone https://www.github.com/farizrahman4u/recurrentshop.git
%cd recurrentshop
!python setup.py install
!pip install git+https://github.com/farizrahman4u/seq2seq.git
from seq2seq.models import SimpleSeq2Seq,AttentionSeq2Seq
import keras

#mecab
!apt install aptitude
!apt-get install swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
import MeCab
%cd
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

!pip install gensim
import gensim
from gensim.models import word2vec


#Google Drive
#Notes. Colaboratory may requires you a permission to access Google Drive.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
%pwd


import numpy as np
import matplotlib.pylab as plt
import math
import re
%matplotlib inline
import copy

In [0]:
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
m=MeCab.Tagger("-d  /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd  -Owakati")

In [0]:
model_wv = word2vec.Word2Vec.load("./model/wiki.model")

In [0]:
ASCII_ZENKAKU_CHARS = (
    u'ａ', u'ｂ', u'ｃ', u'ｄ', u'ｅ', u'ｆ', u'ｇ', u'ｈ', u'ｉ', u'ｊ', u'ｋ',
    u'ｌ', u'ｍ', u'ｎ', u'ｏ', u'ｐ', u'ｑ', u'ｒ', u'ｓ', u'ｔ', u'ｕ', u'ｖ',
    u'ｗ', u'ｘ', u'ｙ', u'ｚ',
    u'Ａ', u'Ｂ', u'Ｃ', u'Ｄ', u'Ｅ', u'Ｆ', u'Ｇ', u'Ｈ', u'Ｉ', u'Ｊ', u'Ｋ',
    u'Ｌ', u'Ｍ', u'Ｎ', u'Ｏ', u'Ｐ', u'Ｑ', u'Ｒ', u'Ｓ', u'Ｔ', u'Ｕ', u'Ｖ',
    u'Ｗ', u'Ｘ', u'Ｙ', u'Ｚ',
    u'！', u'”', u'＃', u'＄', u'％', u'＆', u'’', u'（', u'）', u'＊', u'＋',
    u'，', u'－', u'．', u'／', u'：', u'；', u'＜', u'＝', u'＞', u'？', u'＠',
    u'［', u'￥', u'］', u'＾', u'＿', u'‘', u'｛', u'｜', u'｝', u'～', u'　'
)

ASCII_HANKAKU_CHARS = (
    u'a', u'b', u'c', u'd', u'e', u'f', u'g', u'h', u'i', u'j', u'k',
    u'l', u'm', u'n', u'o', u'p', u'q', u'r', u's', u't', u'u', u'v',
    u'w', u'x', u'y', u'z',
    u'A', u'B', u'C', u'D', u'E', u'F', u'G', u'H', u'I', u'J', u'K',
    u'L', u'M', u'N', u'O', u'P', u'Q', u'R', u'S', u'T', u'U', u'V',
    u'W', u'X', u'Y', u'Z',
    u'!', u'"', u'#', u'$', u'%', u'&', u'\'', u'(', u')', u'*', u'+',
    u',', u'-', u'.', u'/', u':', u';', u'<', u'=', u'>', u'?', u'@',
    u'[', u"¥", u']', u'^', u'_', u'`', u'{', u'|', u'}', u'~', u' '
)

KANA_ZENKAKU_CHARS = (
    u'ア', u'イ', u'ウ', u'エ', u'オ', u'カ', u'キ', u'ク', u'ケ', u'コ',
    u'サ', u'シ', u'ス', u'セ', u'ソ', u'タ', u'チ', u'ツ', u'テ', u'ト',
    u'ナ', u'ニ', u'ヌ', u'ネ', u'ノ', u'ハ', u'ヒ', u'フ', u'ヘ', u'ホ',
    u'マ', u'ミ', u'ム', u'メ', u'モ', u'ヤ', u'ユ', u'ヨ',
    u'ラ', u'リ', u'ル', u'レ', u'ロ', u'ワ', u'ヲ', u'ン',
    u'ァ', u'ィ', u'ゥ', u'ェ', u'ォ', u'ッ', u'ャ', u'ュ', u'ョ',
    u'。', u'、', u'・', u'゛', u'゜', u'「', u'」', u'ー'
)

KANA_HANKAKU_CHARS = (
    u'ｱ', u'ｲ', u'ｳ', u'ｴ', u'ｵ', u'ｶ', u'ｷ', u'ｸ', u'ｹ', u'ｺ',
    u'ｻ', u'ｼ', u'ｽ', u'ｾ', u'ｿ', u'ﾀ', u'ﾁ', u'ﾂ', u'ﾃ', u'ﾄ',
    u'ﾅ', u'ﾆ', u'ﾇ', u'ﾈ', u'ﾉ', u'ﾊ', u'ﾋ', u'ﾌ', u'ﾍ', u'ﾎ',
    u'ﾏ', u'ﾐ', u'ﾑ', u'ﾒ', u'ﾓ', u'ﾔ', u'ﾕ', u'ﾖ',
    u'ﾗ', u'ﾘ', u'ﾙ', u'ﾚ', u'ﾛ', u'ﾜ', u'ｦ', u'ﾝ',
    u'ｧ', u'ｨ', u'ｩ', u'ｪ', u'ｫ', u'ｯ', u'ｬ', u'ｭ', u'ｮ',
    u'｡', u'､', u'･', u'ﾞ', u'ﾟ', u'｢', u'｣', u'ｰ'
)

DIGIT_ZENKAKU_CHARS = (
    u'０', u'１', u'２', u'３', u'４', u'５', u'６', u'７', u'８', u'９'
)

DIGIT_HANKAKU_CHARS = (
    u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9'
)

KANA_TEN_MAP = (
    (u'ガ', u'ｶ'), (u'ギ', u'ｷ'), (u'グ', u'ｸ'), (u'ゲ', u'ｹ'), (u'ゴ', u'ｺ'),
    (u'ザ', u'ｻ'), (u'ジ', u'ｼ'), (u'ズ', u'ｽ'), (u'ゼ', u'ｾ'), (u'ゾ', u'ｿ'),
    (u'ダ', u'ﾀ'), (u'ヂ', u'ﾁ'), (u'ヅ', u'ﾂ'), (u'デ', u'ﾃ'), (u'ド', u'ﾄ'),
    (u'バ', u'ﾊ'), (u'ビ', u'ﾋ'), (u'ブ', u'ﾌ'), (u'ベ', u'ﾍ'), (u'ボ', u'ﾎ'),
    (u'ヴ', u'ｳ')
)

KANA_MARU_MAP = (
    (u'パ', u'ﾊ'), (u'ピ', u'ﾋ'), (u'プ', u'ﾌ'), (u'ペ', u'ﾍ'), (u'ポ', u'ﾎ')
)


ascii_zh_table = {}
ascii_hz_table = {}
kana_zh_table = {}
kana_hz_table = {}
digit_zh_table = {}
digit_hz_table = {}

for (az, ah) in zip(ASCII_ZENKAKU_CHARS, ASCII_HANKAKU_CHARS):
    ascii_zh_table[az] = ah
    ascii_hz_table[ah] = az

for (kz, kh) in zip(KANA_ZENKAKU_CHARS, KANA_HANKAKU_CHARS):
    kana_zh_table[kz] = kh
    kana_hz_table[kh] = kz

for (dz, dh) in zip(DIGIT_ZENKAKU_CHARS, DIGIT_HANKAKU_CHARS):
    digit_zh_table[dz] = dh
    digit_hz_table[dh] = dz

kana_ten_zh_table = {}
kana_ten_hz_table = {}
kana_maru_zh_table = {}
kana_maru_hz_table = {}
for (ktz, kth) in KANA_TEN_MAP:
    kana_ten_zh_table[ktz] = kth
    kana_ten_hz_table[kth] = ktz

for (kmz, kmh) in KANA_MARU_MAP:
    kana_maru_zh_table[kmz] = kmh
    kana_maru_hz_table[kmh] = kmz

del ASCII_ZENKAKU_CHARS, ASCII_HANKAKU_CHARS, \
    KANA_ZENKAKU_CHARS, KANA_HANKAKU_CHARS,\
    DIGIT_ZENKAKU_CHARS, DIGIT_HANKAKU_CHARS,\
    KANA_TEN_MAP, KANA_MARU_MAP

kakko_zh_table = {
    u'｟': u'⦅', u'｠': u'⦆',
    u'『': u'｢', u'』': u'｣',
    u'〚': u'⟦', u'〛': u'⟧',
    u'〔': u'❲', u'〕': u'❳',
    u'〘': u'⟬', u'〙': u'⟭',
    u'《': u'⟪', u'》': u'⟫',
    u'【': u'(', u'】': u')',
    u'〖': u'(', u'〗': u')'
}
kakko_hz_table = {}
for k, v in kakko_zh_table.items():
    kakko_hz_table[v] = k


def zen2han(text="", ascii_=True, digit=True, kana=True, kakko=True, ignore=()):
    result = []
    for c in text:
        if c in ignore:
            result.append(c)
        elif ascii_ and (c in ascii_zh_table):
            result.append(ascii_zh_table[c])
        elif digit and (c in digit_zh_table):
            result.append(digit_zh_table[c])
        elif kana and (c in kana_zh_table):
            result.append(kana_zh_table[c])
        elif kana and (c in kana_ten_zh_table):
            result.append(kana_ten_zh_table[c] + u'ﾞ')
        elif kana and (c in kana_maru_zh_table):
            result.append(kana_maru_zh_table[c] + u'ﾟ')
        elif kakko and (c in kakko_zh_table):
            result.append(kakko_zh_table[c])
        else:
            result.append(c)
    return "".join(result)


def han2zen(text, ascii_=True, digit=True, kana=True, kakko=True, ignore=()):
    result = []
    for i, c in enumerate(text):
        if c == u'ﾞ' or c == u'ﾟ':
            continue
        elif c in ignore:
            result.append(c)
        elif ascii_ and (c in ascii_hz_table):
            result.append(ascii_hz_table[c])
        elif digit and (c in digit_hz_table):
            result.append(digit_hz_table[c])
        elif kana and (c in kana_ten_hz_table) and (text[i + 1] == u'ﾞ'):
            result.append(kana_ten_hz_table[c])
        elif kana and (c in kana_maru_hz_table) and (text[i + 1] == u'ﾟ'):
            result.append(kana_maru_hz_table[c])
        elif kana and (c in kana_hz_table):
            result.append(kana_hz_table[c])
        elif kakko and (c in kakko_hz_table):
            result.append(kakko_hz_table[c])
        else:
            result.append(c)
    return "".join(result)

In [0]:
url = re.compile(r"http(s)?://([\w\-]+\.)+[\w\-]+(\/[\w\- ./?%&=]*)?")
def line2vec(text,length=20):
  #記号類
  text = re.sub("[！!]+","!",text)
  text = re.sub("[？?]+","?",text)
  text = zen2han(text, kana=False)
  text = han2zen(text, ascii_=False, digit=False, kakko=False)
  text = re.sub("\d+","0",text)
  text = text.lower()
  text = re.sub("w+","w",text)
  text = url.sub("",text)
  text = text.replace("[ファイル]","")
  
  #OLK特有の固有名詞、他
  text = text.replace("olk","オリエンテーリング サークル")
  text = text.replace("メンブレ","メンタル ブレイク")
  text = text.replace("ブリケ","ストレージ")
  text = re.sub("オリエンテーリング|オリエン","オリエンテーリング",text)
  text = text.replace("スクショ","スクリーンショット")
  text = text.replace("追いコン","追い出し コンパ")
  
  #人名#この置換処理は個人情報保護の観点から記載していません。

  
  parsed = m.parse(text).strip().split(" ")
  
  vector = [model_wv[p] for p in parsed if p in model_wv]
  if length:
    if length<len(vector):
      vector = vector[len(vector)-length::]
    else:
      for _ in range(length-len(vector)):
        vector.append(model_wv["。"])
  return vector
  
def vec2line(vector):
  text = ""
  for v in vector:
     text += model_wv.most_similar( [ v ], [], 1)[0][0]
  return text
  
def inflation(vector):
  vector += np.random.randn(*vector.shape)*0.1
  return vector

In [0]:
q=[]
a=[]
with open("talk.txt") as fp:
  for line in fp:
    name,text=line.split("\t")
    if name=="":#個人情報保護の観点から削除しています
      a.append(line2vec(text,length=20))
    else:
      q.append(line2vec(text,length=20)[::-1])
q=np.asarray(q)
a=np.asarray(a)

In [0]:
q=q.reshape(-1,20,200)
a=a.reshape(-1,20,200)
q_origin=copy.deepcopy(q)
a_origin=copy.deepcopy(a)

for i in range(10):
  q=np.vstack((q,inflation(q_origin)))
  a=np.vstack((a,inflation(a_origin)))

In [0]:
model = AttentionSeq2Seq(input_dim=200,input_length=20, output_length=20, output_dim=200)

# 学習の設定
model.compile(loss='mse', optimizer='Adam')

In [0]:
model.fit(q, a, epochs=150, batch_size=32)
model.save("er_line_without_inflation.hdf5")

In [0]:
model.load_weights("er.hdf5")

In [0]:
def talk(text):
  vector=np.asarray(line2vec(text,length=20)[::-1])
  vector=vector.reshape(1,20,200)
  predict=model.predict(vector,batch_size=132)
  return vec2line(predict.reshape(20,200))

In [0]:
talk("元気にしてる?")